In [1]:
daily_query="""match (u:User_{0})-[:POSTS]->(t:Tweet_{0}) \
where exists(t.mentions_Trump) \
or exists(t.mentions_Clinton) \
or exists(t.mentions_Sanders) \
or exists(t.mentions_Cruz) \
or exists(t.mentions_Rubio) \
or exists(t.topic) \
WITH "{0}" as day, u.screen_name as username, t.mentions_Trump as Trump, t.mentions_Clinton as Clinton, \
t.mentions_Sanders as Sanders, t.mentions_Cruz as Cruz, t.mentions_Rubio as Rubio, t.topic as topic, \
count(t) as frequency \
RETURN day, username, Trump, Clinton, Sanders, Cruz, Rubio, topic, frequency \
order by day asc, username, frequency desc"""

user_header=['day', 'user', 'supports' 'Trump','Clinton','Sanders','Cruz','Rubio','topic','frequency']

In [2]:
partitions = ['2015_10_11', '2015_10_12', '2015_10_13', '2015_10_14', '2015_10_15',\
              '2016_1_26', '2016_1_27', '2016_1_28', '2016_1_29', '2016_1_30', '2016_2_2', '2016_2_3', \
              '2016_2_4', '2016_2_5', '2016_2_6', '2016_2_7', '2016_2_8', '2016_2_9', '2016_2_10',\
              '2016_2_11', '2016_2_12', '2016_2_13', '2016_2_14', '2016_2_15', \
              '2016_2_23', '2016_2_24', '2016_2_25', '2016_2_26', '2016_2_27', \
              '2016_3_1', '2016_3_2','2016_3_3', '2016_3_4', '2016_3_5', '2016_3_6', '2016_3_7', \
              '2016_3_8', '2016_3_9', '2016_3_10', '2016_3_11']


In [3]:

#set up logging
logger = logging.getLogger('UserResultsGroupedByCandidateTopic')
hdlr = logging.FileHandler('./user_results_remote.log')
formatter = logging.Formatter('%(asctime)s %(levelname)s %(message)s')
hdlr.setFormatter(formatter)
logger.addHandler(hdlr) 
logger.setLevel(logging.INFO)
        

In [4]:
import time
import logging
import csv
from neo4jreader import neo4j_reader

#write daily file
def writedaily(record):
    
    keylist = [record[1],record[2],record[3],record[4],record[5],record[6],record[7]]
    value = record[8]
            
    #some of the neutral items were labeled true instead of neu, correct these.
    #TODO: correct these in the database.
    for n in [1,2,3,4,5]:
        if ("true" in str(keylist[n]).lower()):
            keylist[n]="neu"
            
    return tuple(keylist),value



reader = neo4j_reader('local',logger)

with open('user_results.csv', 'wb') as csvfile:
    
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(user_header)
    
    for partition in partitions:
    #for partition in ['2015_10_11']:

        start = time.time()

        formatted_query = daily_query.format(partition)

        for record in reader.runquery(formatted_query):
            key, value = writedaily(record)

            if (key in userdata):
                userdata[key] = userdata[key] + value
            else:
                userdata[key] = value

        print partition, time.time() - start, len(userdata)

with open('user_results.csv', 'wb') as csvfile:
    
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(user_header)
            
    for k,v in userdata.iteritems():
        outputrow = list(k)
        outputrow.append(v)
        csvwriter.writerow(outputrow)

2015_10_11 3.23056697845 28273
2015_10_12 49.6191000938 307315
2015_10_13 75.3384001255 677483
2015_10_14 150.32089901 1542499
2015_10_15 13.2918889523 1637557
2016_1_26 184.329207897 2290487
2016_1_27 80.2932589054 2917060
2016_1_28 80.0034711361 3411450
2016_1_29 203.162105083 4040148
2016_1_30 73.6138970852 4437150
2016_2_2 122.505506039 5539748
2016_2_3 116.717420816 6120606
2016_2_4 143.415338039 6668643
2016_2_5 148.466931105 7221482
2016_2_6 118.636186123 7603201
2016_2_7 128.264627934 8217531
2016_2_8 63.4797611237 8619053
2016_2_9 73.4079880714 8930879
2016_2_10 211.548989058 9674699
2016_2_11 89.1528251171 10105828
2016_2_12 96.6073479652 10649356
2016_2_13 133.594691038 10990912
2016_2_14 166.461673021 11495185
2016_2_15 103.713423967 11802723
2016_2_23 115.545479059 12298305
2016_2_24 113.117329121 12988575
2016_2_25 209.541116953 13608202
2016_2_26 136.376976967 14400981
2016_2_27 151.339128017 14962068
2016_3_1 387.882022142 15666841
2016_3_2 923.996485949 16754559
2016_3